# Problem 4 Solution

I chose to do problem 4 from the list of screener questions. I didn't look at the qiskit implementation, except to compare the output of my code and that of qiskit.

## Qiskit Circuit
To validate my implementation, first we create a simple circuit in qiskit and run simluation on it.

In [1]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(5, 5)
qc.h(0)
qc.h(1)
qc.cx(1, 0)
qc.cy(3, 1)
qc.cz(0, 4)
qc.h(3)
qc.unitary([[0.5 + 0.5j, 0.5 - 0.5j], [0.5 - 0.5j, 0.5 + 0.5j]], 2)
out = qc.measure([0, 1, 2, 3, 4], [0, 1, 2, 3, 4])
qc.draw()


┌───┐   ┌───┐             ┌─┐      
q_0: ───┤ H ├───┤ X ├──────■──────┤M├──────
        ├───┤   └─┬─┘┌───┐ │      └╥┘┌─┐   
q_1: ───┤ H ├─────■──┤ Y ├─┼───────╫─┤M├───
     ┌──┴───┴──┐ ┌─┐ └─┬─┘ │       ║ └╥┘   
q_2: ┤ unitary ├─┤M├───┼───┼───────╫──╫────
     └─────────┘ └╥┘   │   │ ┌───┐ ║  ║ ┌─┐
q_3: ─────────────╫────■───┼─┤ H ├─╫──╫─┤M├
                  ║        │ └┬─┬┘ ║  ║ └╥┘
q_4: ─────────────╫────────■──┤M├──╫──╫──╫─
                  ║           └╥┘  ║  ║  ║ 
c: 5/═════════════╩════════════╩═══╩══╩══╩═
                  2            4   0  1  3

In [2]:
from qiskit import Aer, execute

def simulate(circuit, num_shots):
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = execute(circuit, backend_sim, shots=num_shots)
    result_sim = job_sim.result()
    return result_sim.get_counts(circuit)

qiskit_counts = simulate(qc, num_shots=10000)

## JQiskit Circuit
Then I built the same circuit in my simulator and ran it.

In [3]:
from jqiskit.api import QuantumCircuit as JQuantumCircuit

qc = JQuantumCircuit(5)
qc.h(0)
qc.h(1)
qc.cx(1, 0)
qc.cy(3, 1)
qc.cz(0, 4)
qc.h(3)
qc.sqrtnot(2)
print(qc)
jqiskit_counts, state = qc.measure(num_shots=10000)

Hadamard: (0,)
Hadamard: (1,)
CX: (1, 0)
CY: (3, 1)
CZ: (0, 4)
Hadamard: (3,)
SQRTNOT: (2,)


## Validation

I then compared the two by looking at the simulation output states. Note how qiskit is the opposite endian-ness as my implementation, so I had to flip the state to do a proper comparison.

In [4]:
assert len(jqiskit_counts) == len(qiskit_counts), "Number of states don't match!"

for state_str in jqiskit_counts:
    print(f'state: {state_str}, qiskit: {qiskit_counts[state_str[::-1]]}; jqiskit: {jqiskit_counts[state_str]}')

state: 10110, qiskit: 641; jqiskit: 588
state: 11000, qiskit: 603; jqiskit: 622
state: 01100, qiskit: 667; jqiskit: 658
state: 10000, qiskit: 584; jqiskit: 667
state: 00000, qiskit: 646; jqiskit: 592
state: 11010, qiskit: 586; jqiskit: 589
state: 00010, qiskit: 595; jqiskit: 686
state: 00110, qiskit: 616; jqiskit: 639
state: 01110, qiskit: 619; jqiskit: 644
state: 11110, qiskit: 674; jqiskit: 621
state: 01010, qiskit: 655; jqiskit: 654
state: 01000, qiskit: 620; jqiskit: 598
state: 10010, qiskit: 618; jqiskit: 639
state: 00100, qiskit: 659; jqiskit: 606
state: 10100, qiskit: 637; jqiskit: 600
state: 11100, qiskit: 580; jqiskit: 597


In [16]:
import numpy as np

qc = JQuantumCircuit(2)
qc.parametric('[[1, sqrt(3)], [0.0 , exp(1.0j*theta)]]', 0)
theta = np.pi / 7
qc.measure(feed_dict={'theta': theta})

SyntaxError: invalid syntax (<string>, line 1)

In [12]:
qc = QuantumCircuit(2, 2)
qc.unitary([[1, 0.], [0.0, np.exp(1.0j*theta)]], 0)
qc.measure([0, 1], [0, 1])
simulate(qc, 10000)

{'00': 10000}